In [1]:
#Notebook for development of the game
# Imports main modules - Player, MapGenerator, etc.
# Implements pygame controls and GUI

In [1]:
import sys, random, os, math
import numpy as np

#where non-dev code is located
sys.path.append('../app')

In [2]:
#install pygame
#!{sys.executable} -m pip install pygame

In [2]:
import pygame

#config stores constants and default settings
from config import Config

#helpful pygame wrapper
import pygame_utils as pyg

pygame 2.0.1 (SDL 2.0.14, Python 3.9.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
# Game GUI consists of:
# - menu through which the user chooses a few settings the starts the game
# - game screen consisting of
#   - the map and players
#   - some type of status bar on the bottom or side allowing restart and showing info
# - winner outcome screen which displays at game end for a while before going back to the menu

In [ ]:
#incremental building of pygame components since they're sensitive and hard to debug

In [6]:
#display icon, title, background, one sprite
pygame.init()
config = Config()

pyg.screenSize(840, 640)
pyg.setBackgroundImage('../app/resources/maps/map_840_640.png')
pyg.setIcon(config.game_icon_image_path)
pyg.setWindowTitle(config.game_name)

blue_flag_sprite = pyg.makeSprite('../app/resources/sprites/blue_flag_20.png')
blue_flag_x_pos = random.randint(config.map_border_size+50, config.screen_width//3)
blue_flag_y_pos = random.randint(config.map_border_size+50, config.screen_height - config.map_border_size)
print(blue_flag_x_pos, blue_flag_y_pos)
pyg.moveSprite(blue_flag_sprite, x=200, y=200)
pyg.showSprite(blue_flag_sprite)

pyg.endWait()

87 410
Press ESC to quit


In [4]:
%tb
#display icon, title, background
#add character, flag, flag area, speed and terrain check

pygame.init()
config = Config()

speed_terr = {v:k for k,v in config.terrain_speeds.items()}

pyg.screenSize(840, 640)
pyg.setBackgroundImage(config.map_default_path)
pyg.setIcon(config.game_icon_image_path)
pyg.setWindowTitle(config.game_name)

speed_arr = np.load(config.map_default_speed_array)

blue_player_sprite = pyg.makeSprite(config.blue_player_sprite_path, 32)
blue_player_x_pos = random.randint(config.map_border_size, config.screen_width//3)
blue_player_y_pos = random.randint(config.map_border_size, config.screen_height - config.map_border_size)
c_idx = math.floor((blue_player_x_pos - config.map_border_size)//config.terrain_tile_size)
r_idx = math.floor((blue_player_y_pos - config.map_border_size)//config.terrain_tile_size)

speed = int(speed_arr[r_idx, c_idx])
print('player is on terrain %s with speed %d' % (speed_terr[speed], speed))

pyg.moveSprite(blue_player_sprite, x=blue_player_x_pos, y=blue_player_y_pos, centre=True)
pyg.showSprite(blue_player_sprite)

blue_flag_sprite = pyg.makeSprite(config.blue_flag_sprite_path)
blue_flag_x_pos = random.randint(config.map_border_size+50, config.screen_width//3)
blue_flag_y_pos = random.randint(config.map_border_size+50, config.screen_height - config.map_border_size)
pyg.drawEllipse(blue_flag_x_pos, blue_flag_y_pos, 100, 100, colour='blue', linewidth=3)
pyg.moveSprite(blue_flag_sprite, x=blue_flag_x_pos, y=blue_flag_y_pos, centre=True)
pyg.showSprite(blue_flag_sprite)

pyg.tick(10)
    
pyg.endWait()

No traceback available to show.


player is on terrain swamp with speed 7
Press ESC to quit


In [ ]:
config = Config()

In [4]:
def setup_screen():
    pyg.screenSize(config.screen_width, config.screen_height)
    pyg.setBackgroundImage(config.map_default_path)
    pyg.setIcon(config.game_icon_image_path)
    pyg.setWindowTitle(config.game_name)
    

def get_not_allowed_tiles(tile_speeds):
    idx = np.where(tile_speeds==0)
    return list(zip(idx[0].tolist(), idx[1].tolist()))


def get_allowed_sprite_init_tiles(tile_speeds, team):
    side = tile_speeds.shape[1]//3
    if team=='blue':
        idx = np.where(tile_speeds[:,:side]>0)
    else:
        idx = np.where(tile_speeds[:,-side:]>0)
    allowed_init_tiles = list(zip(idx[0].tolist(), idx[1].tolist()))
    random.shuffle(allowed_init_tiles)
    return allowed_init_tiles


def make_flag(allowed_sprite_init_tiles, team):
    #make flag sprite
    if team=='blue':
        flag_sprite = pyg.makeSprite(config.blue_flag_sprite_path)
    else:
        flag_sprite = pyg.makeSprite(config.red_flag_sprite_path)
        
    flag_r_idx, flag_c_idx = allowed_sprite_init_tiles.pop()

    flag_x_pos = flag_c_idx * config.terrain_tile_size
    flag_y_pos = flag_r_idx * config.terrain_tile_size
    
    #don't allow flag area to overlap border
    if team=='blue':
        flag_x_pos = max(config.map_border_size + config.flag_area_size//2, flag_x_pos)
        flag_y_pos = max(config.map_border_size + config.flag_area_size//2, flag_y_pos)
        flag_y_pos = min(config.screen_height - config.map_border_size - config.flag_area_size//2, flag_y_pos)
    else:
        flag_x_pos = min(config.screen_width - config.map_border_size - config.flag_area_size//2, flag_x_pos)
        flag_y_pos = max(config.map_border_size + config.flag_area_size//2, flag_y_pos)
        flag_y_pos = min(config.screen_height - config.map_border_size - config.flag_area_size//2, flag_y_pos)
        
    print('flag on c=%d, r=%s, x=%d, y=%d' % (flag_c_idx, flag_r_idx, flag_x_pos, flag_y_pos))

    #make flag area, put sprite on top
    pyg.drawEllipse(flag_x_pos, flag_y_pos, config.flag_area_size, config.flag_area_size, colour=team, linewidth=3)
    pyg.moveSprite(flag_sprite, x=flag_x_pos, y=flag_y_pos, centre=True)
    pyg.showSprite(flag_sprite)
    
    #prevent players from spawning in flag area - remove available spaces within a square of the flag area
    allowed_sprite_init_tiles = remove_flag_area_tiles(allowed_sprite_init_tiles)
    
    return allowed_sprite_init_tiles, flag_sprite
    
    
def remove_flag_area_tiles(allowed_sprite_init_tiles):
    flag_border = config.flag_area_size//config.terrain_tile_size
    new_allowed_sprite_init_tiles = []
    for r,c in allowed_sprite_init_tiles:
        if (flag_c_idx - flag_border < c < flag_c_idx + flag_border) \
        and (flag_r_idx - flag_border < r < flag_r_idx + flag_border):
            print('removed close flag area c,r',c,r)
            continue
        new_allowed_sprite_init_tiles.append((r,c))
        
    return new_allowed_sprite_init_tiles


def make_players(allowed_sprite_init_tiles, team):
    user_player = make_player(allowed_sprite_init_tiles, team='blue', agent=False)
    
    blue_players = []
    for i in range(config.blue_team_size - 1):
        blue_players.append(make_player(allowed_sprite_init_tiles, team='blue'))
    
    red_players = []
    for i in range(config.red_team_size):
        red_players.append(make_player(allowed_sprite_init_tiles, team='red'))
        
    return user_player, blue_players, red_players

        
def make_player(allowed_sprite_init_tiles, team='', agent=True):
    if not agent: #this will be a different sprite
        player_sprite = pyg.makeSprite(config.blue_player_sprite_path, 32)
    elif team=='blue':
        player_sprite = pyg.makeSprite(config.blue_player_sprite_path, 32)
    else:
        player_sprite = pyg.makeSprite(config.red_player_sprite_path, 32)

    player_r_idx, player_c_idx = allowed_sprite_init_tiles.pop()
    
    player_x_pos = player_c_idx * config.terrain_tile_size
    player_y_pos = player_r_idx * config.terrain_tile_size
    print('%s player on r=%s, c=%d, x=%d, y=%d' % (team, player_r_idx, player_c_idx, player_x_pos, player_y_pos))
    
    pyg.moveSprite(player_sprite, x=player_x_pos, y=player_y_pos, centre=True)
    pyg.showSprite(player_sprite)
    
    return player_sprite


def make_sprites(tile_speeds):
    #get blue side non-lake non border tile locations
    allowed_blue_sprite_init_tiles = get_allowed_sprite_init_tiles(tile_speeds, 'blue')
    
    #blue flag and area
    allowed_blue_sprite_init_tiles, blue_flag = make_flag(allowed_blue_sprite_init_tiles, 'blue')
    
    #create user player on blue team
    user_player = make_player(allowed_blue_sprite_init_tiles, team='blue', agent=False)
    
    #create other blue players
    blue_players = []
    for i in range(config.blue_team_size - 1):
        blue_players.append(make_player(allowed_blue_sprite_init_tiles, team='blue'))
    
    #get red side non-lake non border tile locations
    allowed_red_sprite_init_tiles = get_allowed_sprite_init_tiles(tile_speeds, 'red')
    
    #red flag and area
    allowed_red_sprite_init_tiles, red_flag = make_flag(allowed_red_sprite_init_tiles, 'red')
    
    #create red players
    red_players = []
    for i in range(config.red_team_size):
        red_players.append(make_player(allowed_sprite_init_tiles, team='red'))
        
    return blue_flag, red_flag, user_player, blue_players, red_players

In [ ]:
%tb
#display icon, title, background
#add character, flag, flag area, speed and terrain check
#prevent player or flag from spawning in lake
#allow player to move

pygame.init()

setup_screen()

#speed number corresponding to border and tiles
tile_speeds = np.load(config.map_default_speed_array)

#row, col tile indices that can't be walked on, including borders
not_allowed_tiles = get_not_allowed_tiles(tile_speeds)

#make flags and players
blue_flag, red_flag, user_player, blue_players, red_players = make_sprites(tile_speeds)



In [4]:
#allowed movement

speed = int(tile_speeds[player_r_idx, player_c_idx])
speed_terr = {v:k for k,v in config.terrain_speeds.items()}
print('player is on terrain %s with speed %d' % (speed_terr[speed], speed))

prev_blue_player_x_pos, prev_blue_player_y_pos, prev_speed = blue_player_x_pos, blue_player_y_pos, speed

nextFrame = pyg.clock()
frame = 0
while True:
    if pyg.clock() > nextFrame:                         # We only animate our character every 80ms.
        frame = (frame+1)%8                         # There are 8 frames of animation in each direction
        nextFrame += 80                             # so the modulus 8 allows it to loop
    
    if pyg.keyPressed("d"):
        pyg.changeSpriteImage(blue_player_sprite, 0*8+frame)    # 0*8 because right animations are the 0th set in the sprite sheet
        blue_player_x_pos += speed

    elif pyg.keyPressed("s"):
        pyg.changeSpriteImage(blue_player_sprite, 1*8+frame)    # down facing animations are the 1st set
        blue_player_y_pos += speed

    elif pyg.keyPressed("a"):
        pyg.changeSpriteImage(blue_player_sprite, 2*8+frame)    # and so on
        blue_player_x_pos -= speed

    elif pyg.keyPressed("w"):
        pyg.changeSpriteImage(blue_player_sprite, 3*8+frame)
        blue_player_y_pos -= speed

    else:
        pyg.changeSpriteImage(blue_player_sprite, 1 * 8 + 5)  # the static facing front look
        
    #don't allow player on the borders
    if blue_player_x_pos<=config.map_border_size or \
    blue_player_x_pos>=config.screen_width - config.map_border_size or \
    blue_player_y_pos<=config.map_border_size or \
    blue_player_y_pos>=config.screen_height - config.map_border_size:
        speed = 0
    else:
        #what terrain is the player on now?
        c_idx = math.floor((blue_player_x_pos-config.map_border_size)//config.terrain_tile_size)
        r_idx = math.floor((blue_player_y_pos-config.map_border_size)//config.terrain_tile_size)
        speed = int(speed_arr[r_idx, c_idx])
    
    if not speed==prev_speed:
        print('player moved from %s to %s speed %d to %d.' % (speed_terr[prev_speed], speed_terr[speed], prev_speed, speed))

    #if player tried to walk where they're not allowed, revert to prev pos & speed or else
    # they won't be able to leave it
    if not speed:
        blue_player_x_pos, blue_player_y_pos, speed = prev_blue_player_x_pos, prev_blue_player_y_pos, prev_speed
    else:
        pyg.moveSprite(blue_player_sprite, blue_player_x_pos, blue_player_y_pos)
        prev_blue_player_x_pos, prev_blue_player_y_pos, prev_speed = blue_player_x_pos, blue_player_y_pos, speed
    
    pyg.tick(10)
    
pyg.endWait()

No traceback available to show.


flag on r=1, c=1, x=70, y=70
removed close flag area 0 1
removed close flag area 5 2
removed close flag area 0 2
removed close flag area 4 1
removed close flag area 5 1
removed close flag area 0 0
removed close flag area 0 5
removed close flag area 5 0
removed close flag area 2 0
removed close flag area 1 0
removed close flag area 2 1
removed close flag area 4 0
removed close flag area 3 0
removed close flag area 0 4
player on r=4, c=7, x=160, y=100
player is on terrain swamp with speed 7
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to plain speed 7 to 10

player moved from plain to swamp speed 10 to 7.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to plain speed 7 to 10.
player moved from plain to swamp speed 10 to 7.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to l

player moved from swamp to plain speed 7 to 10.
player moved from plain to swamp speed 10 to 7.
player moved from swamp to plain speed 7 to 10.
player moved from plain to swamp speed 10 to 7.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to lake speed 7 to 0.
player moved from swamp to

SystemExit: 

c:\ml\env\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# some game logic

#sounds
#victory magic
grabbed_flag_sound = makeSound("sounds/grabbed_flag.wav")
#bonk, tweet tweet
incapacitated_sound = makeSound('sounds/incapacitated.wav')
#floop
dropped_flag_sound = makeSound('sounds/dropped_flag.wav')
#slap
tagged_sound = makeSound('sounds/tagged.wav')
#bling
revived_sound = makeSound('sounds/revived.wav')

#music
makeMusic("sounds/default.wav")
playMusic(loops=100)


#sprites - flags, flag areas, territory overlays
blue_flag = makeSprite('blue_flag.png')
red_flag = makeSprite('red_flag.png')


#human_player if not observe mode - players wrap sprites for: normal, has flag, is incapacitated, tagging
players = [HumanPlayer('blue', 0)]
players.extend([AgentPlayer('blue', i+1) for i in team_size-1])
players.extend([AgentPlayer('red', i) for i in team_size])


flag_in_play = True

game_winner=''

while True:
    if game_winner:
        #hide all sprites
        hideAll()
        if game_winner=='blue':
            setBackgroundImage(blue_wins_background)
            stopMusic()
            makeMusic("sounds/blue_victory.wav")
            playMusic(loops=1)
        else:
            setBackgroundImage(red_wins_background)
            stopMusic()
            makeMusic("sounds/red_victory.wav")
            playMusic(loops=1)
        pause(5000)
        break
        
    #quit?
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    
    #process players
    for player in players:

        #what is the player touching
        player_touching = allTouching(player.sprite)
        if player_touching:
            for thing in player_touching:
                #how check type?
                
                #touched flag area and has flag - won
                if player.has_flag and type(thing)==type(blue_flag_area) and player.team=='blue' \
                or type(thing)==type(red_flag_area) and player.team=='red': 
                    game_winner=player.team
                    break
                    
                #touched a flag
                if type(thing)==type(red_flag) and player.team=='blue' or type(thing)==type(blue_flag) and player.team=='red': 
                    #play sound
                    playSound(grabbed_flag_sound)

                    #remove flag
                    if player.team=='blue'
                        killSprite(red_flag)
                    else:
                        killSprite(blue_flag)

                    #remove player without flag
                    killSprite(player.sprite)

                    #draw player with flag
                    player.sprite = player.has_flag_sprite
                    showSprite(player.sprite)

                    #if a flag not already in play, play tense music
                    if not flag_in_play:
                        stopMusic()
                        makeMusic("sounds/flag_runner.wav")
                        playMusic(loops=100)
                        flag_in_play = True
                
                #touched a player - helping or tagging?
                elif type(thing)==type(player): 
                    
                    #helping
                    if player.team==thing.team:
                        playSound(revived_sound)
                        thing.energy = 10
                        player.energy = 10
                    
                    #tagging - pt on opponent side  
                    elif thing.has_flag and thing.energy>0:
                        playSound(dropped_flag_sound)
                        thing.energy = 0
                        
                        thing.sprite = thing.incapacitated_sprite
                        showSprite(player.sprite)
            
                        #drop the flag
                        if thing.team=='blue':
                        #TODO make sure it isn't dropped off the map
                            moveSprite(blue_flag, thing.x + 40, thing.y + 40)
                            showSprite(blue_flag)
                        else:
                            moveSprite(red_flag, thing.x + 40, thing.y + 40)
                            showSprite(red_flag)
                            
                    #tagging - pt on opponent side  
                    elif thing.on_opponent_side and thing.energy>0:
                        playSound(tagged_sound)
                        thing.energy = 0

In [ ]:
pygame.init()
config = Config()

try:
    pyg.screenSize(config.screen_width, config.screen_height)
    pyg.setBackgroundImage(config.map_default_path)
    pyg.setIcon(config.game_icon_image_path)
    pyg.setWindowTitle(config.game_name)

    border = config.map_border_size
    tile_size = config.terrain_tile_size
    terr_speeds = config.terrain_speeds

    speed_arr = np.load(config.map_default_speed_array)

    blueSprite  = pyg.makeSprite(config.blue_player_sprite_path, 32)  # links.gif contains 32 separate frames of animation.

    x_pos = random.randint(border,config.screen_width//3)
    y_pos = random.randint(border,config.screen_height-border)

    c_idx = math.floor((x_pos-border)//tile_size)
    r_idx = math.floor((y_pos-border)//tile_size)

    speed = int(speed_arr[r_idx, c_idx])
    speed_terr = {v:k for k,v in terr_speeds.items()}
    print(x_idx, y_idx, speed, speed_terr[speed])

    pyg.moveSprite(blueSprite, x=x_pos, y=y_pos, centre=True)
    pyg.showSprite(blueSprite)

    nextFrame = pyg.clock()
    frame = 0
    i=0
    while True:
        i+=1
        if i>1000:
            print('times up')
            break

        if pyg.clock() > nextFrame:                         # We only animate our character every 80ms.
            frame = (frame+1)%8                         # There are 8 frames of animation in each direction
            nextFrame += 80                             # so the modulus 8 allows it to loop

        c_idx = math.floor((x_pos-border)/tile_size)
        r_idx = math.floor((y_pos-border)/tile_size)
        #print(x_pos, y_pos, x_idx, y_idx)
        speed = speed_arr[r_idx, c_idx]

        if pyg.keyPressed("d"):
            pyg.changeSpriteImage(blueSprite, 0*8+frame)    # 0*8 because right animations are the 0th set in the sprite sheet
            x_pos += speed

        elif pyg.keyPressed("s"):
            pyg.changeSpriteImage(blueSprite, 1*8+frame)    # down facing animations are the 1st set
            y_pos += speed

        elif pyg.keyPressed("a"):
            pyg.changeSpriteImage(blueSprite, 2*8+frame)    # and so on
            x_pos -= speed

        elif pyg.keyPressed("w"):
            pyg.changeSpriteImage(blueSprite, 3*8+frame)
            y_pos -= speed

        else:
            pyg.changeSpriteImage(blueSprite, 1 * 8 + 5)  # the static facing front look

        pyg.moveSprite(blueSprite, x_pos, y_pos)

        pyg.tick(120)
except Exception as ex:
    print(ex)
    pyg.killSprite(blueSprite)
    pyg.endWait()

2 15 10 plain


In [ ]:
#untested code skeleton
#class for menu GUI elements

class Menu():
    def __init__(self, config, map_generator):
        self.config = config
        
        #holds a generated map, can replace generated map with .generate_map()
        self.map_generator = map_generator
        
        #the dict of values populated from the menu
        self.menu_values = {}
        
        self.menu_background = pyg.makeSprite(config.menu_background_image_path)
        pyg.moveSprite(self.menu_background, 200,150) #TODO - location
        pyg.showSprite(self.menu_background)
            
        self.text_label = pyg.makeLabel("Capture the Flag", 22, 200, 50, "white", font="impact")#TODO - location
        pyg.showLabel(self.text_label)
        
        #elements of the menu
        self.game_mode_select = pyg.makeSprite(config.menu_game_mode_select_image_path)
        pyg.moveSprite(self.game_mode_select, 200,150)#TODO - location
        pyg.showSprite(self.game_mode_select)
        
        self.team_size_select = pyg.makeSprite(config.menu_team_size_select_image_path)
        pyg.moveSprite(self.team_size_select, 200,150)#TODO - location
        pyg.showSprite(self.team_size_select)
            
        self.regenerate_map_button = pyg.makeSprite(config.menu_regenerate_map_button_image_path)
        pyg.moveSprite(self.regenerate_map_button, 200,150)#TODO - location
        pyg.showSprite(self.regenerate_map_button)
            
        self.start_game_button = pyg.makeSprite(config.menu_start_game_button_image_path)
        pyg.moveSprite(self.start_game_button, 200,150)#TODO - location
        pyg.showSprite(self.start_game_button)
            
        #display small version of generated map so user can choose to regenerate it
        #TODO use the menu_image in map_generator to paste on the sprite
        self.map_display = pyg.makeSprite(config.menu_map_display_image_path)
        pyg.moveSprite(self.map_display, 200,150)#TODO - location
        pyg.showSprite(self.map_display)
        
    
    def get_values(self):
        '''When Start Game button is pressed, returns dict of values populated from the menu.'''
        game_mode = self.config.game_mode_play
        team_size = self.config.team1_size
        player_team = self.config.team1_name
        
        while True:
            #display menu for choosing game mode, number of players, team, map
            if pyg.spriteClicked(self.game_mode_select):
                self.menu_values['game_mode'] = self.get_game_mode()
                
            elif pyg.spriteClicked(self.team_size_select):
                self.menu_values['team_size'] = self.get_team_size()
                
            elif pyg.spriteClicked(self.player_team_select):
                self.menu_values['player_team'] = self.get_player_team()
                
            elif pyg.spriteClicked(self.regenerate_map_button):
                self.map_generator.generate_map()
                self.update_map(self.map_generator.menu_image)
                
            elif pyg.spriteClicked(self.start_game_button):
                #destroy the menu and return the values
                pyg.hideLabel(self.text_label)
                pyg.killSprite(self.game_mode_select)
                pyg.killSprite(self.team_size_select)
                pyg.killSprite(self.regenerate_map_button)
                pyg.killSprite(self.start_game_button)
                pyg.killSprite(self.map_display)
                pyg.killSprite(self.menu_background)
                return self.menu_values
            
            tick(config.menu_frame_rate)
            
                
    def get_game_mode(self):
        #TODO get from selection
        return self.config.game_mode_play
    
    def get_team_size(self):
        #TODO get from selection
        return self.config.team1_size
    
    def get_player_team(self):
        #TODO get from selection
        return self.config.team1_name
    
    def update_map(self, im):
        #TODO update map image sprite
        pass

In [ ]:
#untested code skeleton
# class for game status window next to map

class GameStatusWindow():
    def __init__(self):
        pass
    def display(self):
        pass

In [ ]:
#untested code skeleton
#class for map GUI elements

class GameMapWindow():
    def __init__(self, game_status_window):
        pass
    def display(self):
        pass

In [ ]:
#untested code skeleton
#top class for creating all GUI elements and running the game

class Game():
    def __init__(self, config):
        self.config = config
        
        #instantiate pygame objects using game vars from config
        pyg.screenSize(config.screen_width, config.screen_height)
        
        pyg.setIcon(config.game_icon_image_path)
        pyg.setWindowTitle(config.game_name)
        pyg.setBackgroundImage(config.game_background_image_path)
        
        # menu
        
        #map may be regenerated in menu, so access map_generator through menu object
        map_generator = MapGenerator()
        map_generator.generate_map()
        
        self.menu = Menu(config, map_generator)
        
        
    def play(self):
        #main game loop
        while True:
            #wait for values from the menu - currently includes game_mode, team_size, player_team
            menu_values = self.menu.get_values()
            game_mode = menu_values['game_mode']
            team_size = menu_values['team_size']
            player_team = menu_values['player_team']
            
            #the generated map is accessible through the map generator in menu
            #update background with generated map
            #self.menu.map_generator
            
            #create player and agents
            
            #update sprites according to their actions
            
            #check for win

            pyg.tick(config.time_delay)
            
        pyg.endWait()

In [ ]:
config = Config()
game = Game(config)
game.play()